In [1]:
%load_ext autoreload

In [77]:
import sys
sys.path.append("..")
from utils import get_hf_dataset
dataset = get_hf_dataset()
dataset

Found cached dataset json (C:/Users/Dell/.cache/huggingface/datasets/json/default-aef40b4b132ab43c/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset json (C:/Users/Dell/.cache/huggingface/datasets/json/default-f4584a93bbbd7e8a/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\Dell\.cache\huggingface\datasets\json\default-aef40b4b132ab43c\0.0.0\fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e\cache-bd9e74115e31da9a.arrow
Loading cached processed dataset at C:\Users\Dell\.cache\huggingface\datasets\json\default-f4584a93bbbd7e8a\0.0.0\fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e\cache-31308b93ec420624.arrow


DatasetDict({
    train: Dataset({
        features: ['annotations', 'text'],
        num_rows: 10995
    })
    test: Dataset({
        features: ['annotations', 'text'],
        num_rows: 1074
    })
})

In [78]:
from utils import parse_to_ner
dataset_ner = parse_to_ner(dataset) 

Loading cached processed dataset at C:\Users\Dell\.cache\huggingface\datasets\json\default-aef40b4b132ab43c\0.0.0\fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e\cache-5274ff6e7b61f37b.arrow
Loading cached processed dataset at C:\Users\Dell\.cache\huggingface\datasets\json\default-f4584a93bbbd7e8a\0.0.0\fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e\cache-bd2f0917eeea4537.arrow


In [79]:
from utils import cast_ner_labels_to_int
casted = cast_ner_labels_to_int(dataset_ner['train'])

Loading cached processed dataset at C:\Users\Dell\.cache\huggingface\datasets\json\default-aef40b4b132ab43c\0.0.0\fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e\cache-c3fc1fbe7918877a.arrow


In [80]:
dataset['train'][0]

{'annotations': [{'from_name': 'label',
   'id': 'C8HPTIM1',
   'to_name': 'text',
   'type': 'labels',
   'value': {'end': 103,
    'labels': ['ORG'],
    'start': 90,
    'text': 'Hongkong Bank'}},
  {'from_name': 'label',
   'id': 'KOWE3RAM',
   'to_name': 'text',
   'type': 'labels',
   'value': {'end': 278,
    'labels': ['ORG'],
    'start': 267,
    'text': 'Rahul & Co.'}}],
 'text': "\n\n(7) On specific query by the Bench about an entry of Rs. 1,31,37,500 on deposit side of Hongkong Bank account of which a photo copy is appearing at p. 40 of assessee's paper book, learned authorised representative submitted that it was related to loan from broker, Rahul & Co. on the basis of his submission a necessary mark is put by us on that photo copy."}

In [81]:
#spacy train data
from utils import get_hf_dataset, parse_to_ner, cast_ner_labels_to_int

dataset = get_hf_dataset()

TRAIN_DATA = []

for example in dataset['train']:
    text = example['text']
    entities = []
    for annotation in example['annotations']:
        result = annotation['value']
        start = result['start']
        end = result['end']
        label = result['labels'][0]
        entities.append((start, end, label))
    TRAIN_DATA.append((text, {"entities": entities}))

TRAIN_DATA[0]

Found cached dataset json (C:/Users/Dell/.cache/huggingface/datasets/json/default-aef40b4b132ab43c/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset json (C:/Users/Dell/.cache/huggingface/datasets/json/default-f4584a93bbbd7e8a/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\Dell\.cache\huggingface\datasets\json\default-aef40b4b132ab43c\0.0.0\fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e\cache-bd9e74115e31da9a.arrow
Loading cached processed dataset at C:\Users\Dell\.cache\huggingface\datasets\json\default-f4584a93bbbd7e8a\0.0.0\fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e\cache-31308b93ec420624.arrow


("\n\n(7) On specific query by the Bench about an entry of Rs. 1,31,37,500 on deposit side of Hongkong Bank account of which a photo copy is appearing at p. 40 of assessee's paper book, learned authorised representative submitted that it was related to loan from broker, Rahul & Co. on the basis of his submission a necessary mark is put by us on that photo copy.",
 {'entities': [(90, 103, 'ORG'), (267, 278, 'ORG')]})

In [82]:
from pathlib import Path

model = "en_core_web_sm"
output_dir=Path("../spacy/output")
n_iter=100

In [83]:
import spacy

nlp = spacy.load(model)  
print("Loaded model '%s'" % model)

Loaded model 'en_core_web_sm'


In [84]:
ner = nlp.get_pipe('ner')

In [85]:
len(ner.labels)

18

In [86]:
import random
from tqdm import tqdm

for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

ner.labels

('CARDINAL',
 'CASE_NUMBER',
 'COURT',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'JUDGE',
 'LANGUAGE',
 'LAW',
 'LAWYER',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'OTHER_PERSON',
 'PERCENT',
 'PERSON',
 'PETITIONER',
 'PRECEDENT',
 'PRODUCT',
 'PROVISION',
 'QUANTITY',
 'RESPONDENT',
 'STATUTE',
 'TIME',
 'WITNESS',
 'WORK_OF_ART')

In [87]:
# print all labels from traindata
labels = set()
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        labels.add(ent[2])
labels

{'CASE_NUMBER',
 'COURT',
 'DATE',
 'GPE',
 'JUDGE',
 'LAWYER',
 'ORG',
 'OTHER_PERSON',
 'PETITIONER',
 'PRECEDENT',
 'PROVISION',
 'RESPONDENT',
 'STATUTE',
 'WITNESS'}

In [88]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [89]:
from spacy.training.example import Example

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.create_optimizer()
    for itn in tqdm(range(n_iter)):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example],
                drop=0.5,
                sgd=optimizer,
                losses=losses,
                
            )
        print(losses)

  0%|          | 0/100 [06:39<?, ?it/s]


KeyboardInterrupt: 